In [1]:
import json
import os
import glob
import re
import pandas as pd
from os import makedirs
from os.path import join, exists
from pathlib import Path
from datetime import datetime, date, timedelta
from pathlib import Path

In [39]:
def combine_data(year, cand, path, target_path, include_hd):
    
    tw_data = pd.read_csv(join(path, cand + '_twitter.csv'))
    tn_data = pd.read_csv(join(path,  cand + '_trends.csv'))
    df = []
    
    if include_hd == True:
        hd_data = pd.read_csv(join(path, cand + '_headlines.csv'))
        df = pd.merge(hd_data, tw_data, how='right', on=['date', 'date'], suffixes=('_h', '_t'))
        df = pd.merge(tn_data, df, how='right', on=['date', 'date'], suffixes=('_tn', '_a'))
        
        df = drop_unnamed_columns(df)
        df = df.fillna(df.mean())
        
        print(df.columns)
        
        df['score']=df['score'].replace(0, df['score'].mean())
        df['score_mn']=df['score_mn'].replace(0, df['score_mn'].mean())
        
        col = df.loc[: , df.columns[2]: df.columns[len(df.columns) - 1]]
        df['score_mean'] = col.mean(axis=1)
        df['final_score'] = df[df.columns[1]] * df['score_mean']
    else:
        #df = pd.merge(hd_data, tw_data, how='right', on=['date', 'date'], suffixes=('_h', '_t'))
        df = pd.merge(tn_data, tw_data, how='right', on=['date', 'date'])
        
        df = drop_unnamed_columns(df)
        df = df.fillna(df.mean())
        df['final_score'] = df[df.columns[1]] * df[df.columns[2]]
    
    df2 = df[['date', 'final_score']]
    
    os.makedirs(target_path, exist_ok=True)
    
    df2.to_csv(join(target_path, cand + '_combined.csv'), index=False)
    

In [40]:
def drop_unnamed_columns(df):
    cols = df.columns
    for c in cols:
        if c == 'Unnamed: 0' or c == 'Unnamed: 0.1' or c == 'Unnamed: 0_x' or c == 'Unnamed: 0_a' \
        or c == 'Unnamed: 0_y' or c == 'headline' or c == 'Unnamed: 0_tn':
            df = df.drop([c], axis=1)
    
    return df

In [41]:
def combine_all_news_hd(cand, path):
    hd_c_data = pd.read_csv(join(path, cand + '_headlines_cnn.csv'))
    hd_c_data = drop_unnamed_columns(hd_c_data)
    hd_f_data = pd.read_csv(join(path, cand + '_headlines_fox-news.csv'))
    hd_f_data = drop_unnamed_columns(hd_f_data)
    #hd_g_data = pd.read_csv(join(path, cand + '_headlines_guardian.csv'))
    #hd_g_data = drop_unnamed_columns(hd_g_data)
    
    
    df = pd.merge(hd_c_data, hd_f_data, how='inner', on=['date', 'date'], suffixes=('_c', '_f'))
    #df = pd.merge(hd_g_data, df, how='right', on=['date', 'date'], suffixes=('_g', '_a'))
    
    df = df.fillna(df.mean())
    
    col = df.loc[: , "score_c":"score_f"]
    df['score_mn'] = col.mean(axis=1)
    
    df2 = df[['date', 'score_mn']]
    
    df2.to_csv(join(path, cand + '_headlines.csv'), index=False)

In [42]:
combine_data('2012', 'obama', join('data', 'candidate', '2012'), join('data2', 'candidate', '2012'), False)
combine_data('2012', 'romney', join('data', 'candidate', '2012'), join('data2', 'candidate', '2012'), False)

In [43]:
combine_data('2016', 'trump', join('data', 'candidate', '2016'), join('data2', 'candidate', '2016'), False)
combine_data('2016', 'hillary', join('data', 'candidate', '2016'), join('data2', 'candidate', '2016'), False)

In [44]:
combine_all_news_hd('trump', join('data', 'candidate', '2020'))
combine_all_news_hd('biden', join('data', 'candidate', '2020'))

In [45]:
combine_data('2020', 'trump', join('data', 'candidate', '2020'), join('data2', 'candidate', '2020'), True)
combine_data('2020', 'biden', join('data', 'candidate', '2020'), join('data2', 'candidate', '2020'), True)

Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')


In [46]:
for tp in ['economy', 'environment', 'health', 'immigration', 'party']:
    combine_all_news_hd('trump', join('data', tp))
    combine_all_news_hd('biden', join('data', tp))
    
    combine_data('2020', 'trump', join('data', tp), join('data2', tp), True)
    combine_data('2020', 'biden', join('data', tp), join('data2', tp), True)

Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
Index(['date', 'score', 'score_mn', 'scores'], dtype='object')
